### Imports

In [54]:
import os
import sys
import json
from urllib.parse import urlparse



In [55]:
sys.path.append(os.path.abspath('..'))  # adds the parent folder to sys.path
from src.utils import request_utils
from src.utils.path_utils import find_project_root,Path

### Configurations

In [56]:
# parameters
base_dir='../'
config_path=None

In [57]:

if config_path is not None:
    BASE_DIR = Path(base_dir)
    config_path
else:
    # fallback: find project root and load config.json
    BASE_DIR = find_project_root()
    config_path = BASE_DIR / 'configurations' / 'config.json'

with open(config_path, 'r', encoding='utf-8') as f:
    config = json.load(f)

print(f"Loaded config from {config_path}")

Loaded config from /Users/vaishnavi/Desktop/Research/DataEngineering/health-insurance-rate-data-etl/configurations/config.json


In [58]:
config_data=request_utils.read_api_config(BASE_DIR/config['api_config_path'])
rate_api_url=config_data['apis']['health_insurance_rate_puf']['endpoint']
meta_data = request_utils.make_api_call(rate_api_url)
with open(BASE_DIR/config['metadata_path'],'w') as meta_file:
    json.dump(meta_data.json(),meta_file,indent=2)


In [59]:
# extract download link and format 
with open(BASE_DIR/config['metadata_path'],'r') as meta_file:
    meta_data=json.load(meta_file)
    format=meta_data['distribution'][0]['format']
    data_url=meta_data['distribution'][0]['downloadURL']

data_url




'https://data.healthcare.gov/datafile/py2025/Rate_PUF.csv'

### API call to download the file

In [60]:
# make API call 
data_response = request_utils.make_api_call(data_url)

#Extract filename from URL
parsed_url = urlparse(data_url)
downloaded_file_name = os.path.basename(parsed_url.path)  # gets 'Rate_PUF.csv'

# Step 3:Save initial Raw file
if data_response.status_code == 200:
    raw_csv_path = os.path.join(BASE_DIR,config["download_dir"], downloaded_file_name)
    with open(raw_csv_path, "wb") as f:
        f.write(data_response.content)
    print(f"✅ File downloaded successfully. Saved raw CSV at {raw_csv_path}")
else:
    raise Exception(f"❌ Failed to download file. Status: {data_response.status_code}")


✅ File downloaded successfully. Saved raw CSV at /Users/vaishnavi/Desktop/Research/DataEngineering/health-insurance-rate-data-etl/data/bronze/Rate_PUF.csv
